Human drug label data was downloaded from [OpenFDA](https://open.fda.gov/data/downloads/) as a set of 12 zipped JSON files.

This is a much smaller download than getting the full release from [Daily Med](https://dailymed.nlm.nih.gov/dailymed/spl-resources-all-drug-labels.cfm).

In [1]:
import zipfile
import json
import pandas as pd
import os
from collections import Counter

def load_data_from_zip(my_zip_file):
    archive = zipfile.ZipFile(DATA_DIR + my_zip_file, 'r')
    my_json_file = my_zip_file.replace('.zip', '')
    return json.loads(archive.read(my_json_file).decode("utf-8"))


def flatten_list(my_list):
    # flatten a list into a single string, if it is a list.
    try:
        flattened = '::'.join(my_list)
    except:
        print(r"ERROR flattening {my_list}")
        flattened = my_list
    return flattened


def extract_dataframe(results):
    data_rows = []
    for res in results:
        row = {}
        # attributes from openfda
        for attr in ['brand_name', 'generic_name', 'product_type']:
            if attr in res['openfda']:
                row[attr] = flatten_list(res['openfda'][attr])
            else:
                row[attr] = ''
        # top level attributes
        for attr in ['description', 'indications_and_usage', 'mechanism_of_action']:
            if attr in res:
                row[attr] = flatten_list(res[attr])
            else:
                row[attr] = ''

        data_rows.append(row)

    return pd.DataFrame(data_rows)


DATA_DIR = '/Users/rmhorton/Downloads/openFDA/'

ZIP_FILES = sorted([f for f in os.listdir(DATA_DIR) if f.endswith('.zip')])
ZIP_FILES

['drug-label-0001-of-0012.json.zip',
 'drug-label-0002-of-0012.json.zip',
 'drug-label-0003-of-0012.json.zip',
 'drug-label-0004-of-0012.json.zip',
 'drug-label-0005-of-0012.json.zip',
 'drug-label-0006-of-0012.json.zip',
 'drug-label-0007-of-0012.json.zip',
 'drug-label-0008-of-0012.json.zip',
 'drug-label-0009-of-0012.json.zip',
 'drug-label-0010-of-0012.json.zip',
 'drug-label-0011-of-0012.json.zip',
 'drug-label-0012-of-0012.json.zip']

In [2]:
labels = load_data_from_zip("drug-label-0001-of-0012.json.zip")
len(labels) # 2 keys, 'meta' and 'results'
labels['meta']

{'disclaimer': 'Do not rely on openFDA to make decisions regarding medical care. While we make every effort to ensure that data is accurate, you should assume all results are unvalidated. We may limit or otherwise restrict your access to the API in line with our Terms of Service.',
 'terms': 'https://open.fda.gov/terms/',
 'license': 'https://open.fda.gov/license/',
 'last_updated': '2024-11-12',
 'results': {'skip': 0, 'limit': 20000, 'total': 235810}}

In [3]:
len(labels['results']) # 20000
list(labels['results'][0].keys())

['spl_product_data_elements',
 'boxed_warning',
 'indications_and_usage',
 'dosage_and_administration',
 'dosage_forms_and_strengths',
 'dosage_forms_and_strengths_table',
 'contraindications',
 'warnings_and_cautions',
 'adverse_reactions',
 'adverse_reactions_table',
 'drug_interactions',
 'use_in_specific_populations',
 'pregnancy',
 'pediatric_use',
 'geriatric_use',
 'overdosage',
 'description',
 'clinical_pharmacology',
 'mechanism_of_action',
 'pharmacodynamics',
 'pharmacokinetics',
 'nonclinical_toxicology',
 'carcinogenesis_and_mutagenesis_and_impairment_of_fertility',
 'clinical_studies',
 'clinical_studies_table',
 'how_supplied',
 'how_supplied_table',
 'information_for_patients',
 'package_label_principal_display_panel',
 'set_id',
 'id',
 'effective_time',
 'version',
 'openfda']

In [4]:
# The top level information is in the 'openfda' object
for res in labels['results'][0:10]:
    key = 'openfda' # 'spl_product_data_elements' 'mechanism_of_action' 'openfda' 'package_label_principal_display_panel'  'description' 'information_for_patients'
    if key in res:
        print(key, ': ', res[key])
    # print(res['id'], "\t", res['description'], "\n")

openfda :  {'application_number': ['ANDA209010'], 'brand_name': ['Amlodipine and Olmesartan Medoxomil'], 'generic_name': ['AMLODIPINE AND OLMESARTAN MEDOXOMIL'], 'manufacturer_name': ['ScieGen Pharmaceuticals, Inc'], 'product_ndc': ['50228-365', '50228-366', '50228-367', '50228-368'], 'product_type': ['HUMAN PRESCRIPTION DRUG'], 'route': ['ORAL'], 'substance_name': ['AMLODIPINE BESYLATE', 'OLMESARTAN MEDOXOMIL'], 'rxcui': ['730861', '730866', '730869', '730872'], 'spl_id': ['1666c9c9-9d30-4155-e063-6394a90afde9'], 'spl_set_id': ['c531fceb-09aa-492b-a5a3-2b004e9eadda'], 'package_ndc': ['50228-365-30', '50228-365-90', '50228-365-10', '50228-366-30', '50228-366-90', '50228-366-10', '50228-367-30', '50228-367-90', '50228-367-10', '50228-368-30', '50228-368-90', '50228-368-10'], 'is_original_packager': [True], 'upc': ['0350228367103', '0350228367301', '0350228366106', '0350228365109', '0350228368100', '0350228366908', '0350228365307', '0350228366304', '0350228368902', '0350228365901', '0350

In [5]:
# Process all zip files
import re

human_prescription_drugs = []
for zfile in ZIP_FILES:
    print(zfile)
    file_number = int(re.sub(r"drug-label-([0-9]+).*", r"\1", zfile))
    labels = load_data_from_zip(zfile)
    results_df = extract_dataframe(labels['results'])
    human_prescription_drugs.append(results_df[results_df['product_type'] == 'HUMAN PRESCRIPTION DRUG'])
    excel_file = f"drug_label_{file_number:02d}.xlsx"
    results_df.to_excel(excel_file)

drug-label-0001-of-0012.json.zip
drug-label-0002-of-0012.json.zip
drug-label-0003-of-0012.json.zip
drug-label-0004-of-0012.json.zip
drug-label-0005-of-0012.json.zip
drug-label-0006-of-0012.json.zip
drug-label-0007-of-0012.json.zip
drug-label-0008-of-0012.json.zip
drug-label-0009-of-0012.json.zip
drug-label-0010-of-0012.json.zip
drug-label-0011-of-0012.json.zip
drug-label-0012-of-0012.json.zip


In [6]:
human_prescription_drugs_df = pd.concat(human_prescription_drugs).reset_index(drop=True)
human_prescription_drugs_df.to_excel("human_prescription_drugs.xlsx")
human_prescription_drugs_df

,brand_name,generic_name,product_type,description,indications_and_usage,mechanism_of_action
0,Amlodipine and Olmesartan Medoxomil,AMLODIPINE AND OLMESARTAN MEDOXOMIL,HUMAN PRESCRIPTION DRUG,11 DESCRIPTION Amlodipine and olmesartan medox...,1 INDICATIONS AND USAGE Amlodipine and olmesar...,12.1 Mechanism of Action Amlodipine and olmesa...
1,ARTHROTEC,DICLOFENAC SODIUM AND MISOPROSTOL,HUMAN PRESCRIPTION DRUG,11 DESCRIPTION ARTHROTEC is a combination prod...,1 INDICATIONS AND USAGE ARTHROTEC is indicated...,12.1 Mechanism of Action ARTHROTEC is a combin...
2,"Polyethylene Glycol 3350, Sodium Chloride, Sod...","POLYETHYLENE GLYCOL 3350, SODIUM CHLORIDE, SOD...",HUMAN PRESCRIPTION DRUG,11 DESCRIPTION For oral solution: Each 5 liter...,1 INDICATIONS AND USAGE Polyethylene glycol 33...,
3,Irbesartan and Hydrochlorothiazide,IRBESARTAN AND HYDROCHLOROTHIAZIDE,HUMAN PRESCRIPTION DRUG,11 DESCRIPTION Irbesartan and Hydrochlorothiaz...,1 INDICATIONS AND USAGE Irbesartan and hydroch...,12.1 Mechanism of Action Irbesartan Angiotensi...
4,Potassium Phosphates,POTASSIUM PHOSPHATES,HUMAN PRESCRIPTION DRUG,"11 DESCRIPTION Potassium Phosphates Injection,...",1 INDICATIONS AND USAGE Potassium Phosphates I...,12.1 Mechanism of Action Phosphorus in the for...
...,...,...,...,...,...,...
36801,Oxycodone Hydrochloride and Acetaminophen,OXYCODONE HYDROCHLORIDE AND ACETAMINOPHEN,HUMAN PRESCRIPTION DRUG,DESCRIPTION Oxycodone Hydrochloride and Acetam...,INDICATIONS & USAGE Oxycodone and Acetaminophe...,
36802,Anaprox DS,NAPROXEN SODIUM,HUMAN PRESCRIPTION DRUG,"11 DESCRIPTION NAPROSYN (naproxen) tablets, EC...","1 INDICATIONS AND USAGE NAPROSYN Tablets, EC-N...",12.1 Mechanism of Action Naproxen has analgesi...
36803,Hydrocodone Bitartrate and Homatropine Methylb...,HYDROCODONE BITARTRATE AND HOMATROPINE METHYLB...,HUMAN PRESCRIPTION DRUG,11 DESCRIPTION Hydrocodone bitartrate and homa...,1 INDICATIONS AND USAGE Hydrocodone bitartrate...,12.1 Mechanism of Action Hydrocodone Hydrocodo...
36804,"dextroamphetamine saccharate, amphetamine aspa...","DEXTROAMPHETAMINE SACCHARATE, AMPHETAMINE ASPA...",HUMAN PRESCRIPTION DRUG,"11 DESCRIPTION Dextroamphetamine Saccharate, A...",1 INDICATIONS AND USAGE Dextroamphetamine Sacc...,12.1 Mechanism of Action Amphetamines are non-...


# Featurize text columns

In [7]:
import os.path

FEATURIZED_FILE = 'human_prescription_drugs_featurized.parquet'

if os.path.exists(FEATURIZED_FILE):
    human_prescription_drugs_df = pd.read_parquet(FEATURIZED_FILE)
else:  # 31 min on Bob's M3 Mac
    from sentence_transformers import SentenceTransformer
    sentxformer = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
    for text_col in ['description', 'indications_and_usage', 'mechanism_of_action']:
        print(f"featurizing {text_col}")
        human_prescription_drugs_df[text_col + '_vector'] = sentxformer.encode(human_prescription_drugs_df[text_col].values).astype('float32').tolist()
    human_prescription_drugs_df.to_parquet(FEATURIZED_FILE)

In [8]:
human_prescription_drugs_df


,brand_name,generic_name,product_type,description,indications_and_usage,mechanism_of_action,description_vector,indications_and_usage_vector,mechanism_of_action_vector
0,Amlodipine and Olmesartan Medoxomil,AMLODIPINE AND OLMESARTAN MEDOXOMIL,HUMAN PRESCRIPTION DRUG,11 DESCRIPTION Amlodipine and olmesartan medox...,1 INDICATIONS AND USAGE Amlodipine and olmesar...,12.1 Mechanism of Action Amlodipine and olmesa...,"[-0.0251484252512455, -0.014093315228819847, 0...","[-0.04058678075671196, 0.015312662348151207, 0...","[-0.029752785339951515, -0.0015405130106955767..."
1,ARTHROTEC,DICLOFENAC SODIUM AND MISOPROSTOL,HUMAN PRESCRIPTION DRUG,11 DESCRIPTION ARTHROTEC is a combination prod...,1 INDICATIONS AND USAGE ARTHROTEC is indicated...,12.1 Mechanism of Action ARTHROTEC is a combin...,"[0.02258964814245701, -0.02370976097881794, -0...","[0.026246052235364914, -0.0003164233057759702,...","[0.020150907337665558, -0.0012751225149258971,..."
2,"Polyethylene Glycol 3350, Sodium Chloride, Sod...","POLYETHYLENE GLYCOL 3350, SODIUM CHLORIDE, SOD...",HUMAN PRESCRIPTION DRUG,11 DESCRIPTION For oral solution: Each 5 liter...,1 INDICATIONS AND USAGE Polyethylene glycol 33...,,"[0.05875419080257416, -0.01822586916387081, -0...","[0.04296257719397545, -0.02611582726240158, -0...","[-0.012503336183726788, 0.06143886223435402, -..."
3,Irbesartan and Hydrochlorothiazide,IRBESARTAN AND HYDROCHLOROTHIAZIDE,HUMAN PRESCRIPTION DRUG,11 DESCRIPTION Irbesartan and Hydrochlorothiaz...,1 INDICATIONS AND USAGE Irbesartan and hydroch...,12.1 Mechanism of Action Irbesartan Angiotensi...,"[0.018642738461494446, -0.061663269996643066, ...","[0.01213595550507307, -0.013446979224681854, 0...","[0.006609566509723663, -0.045228831470012665, ..."
4,Potassium Phosphates,POTASSIUM PHOSPHATES,HUMAN PRESCRIPTION DRUG,"11 DESCRIPTION Potassium Phosphates Injection,...",1 INDICATIONS AND USAGE Potassium Phosphates I...,12.1 Mechanism of Action Phosphorus in the for...,"[0.029501350596547127, -0.08191590756177902, -...","[0.03445522114634514, -0.04430317133665085, 0....","[-0.007063195109367371, 0.019696714356541634, ..."
...,...,...,...,...,...,...,...,...,...
36801,Oxycodone Hydrochloride and Acetaminophen,OXYCODONE HYDROCHLORIDE AND ACETAMINOPHEN,HUMAN PRESCRIPTION DRUG,DESCRIPTION Oxycodone Hydrochloride and Acetam...,INDICATIONS & USAGE Oxycodone and Acetaminophe...,,"[-0.0014800369972363114, 0.023205861449241638,...","[-0.0018607164965942502, -0.01076608058065176,...","[-0.012503336183726788, 0.06143886223435402, -..."
36802,Anaprox DS,NAPROXEN SODIUM,HUMAN PRESCRIPTION DRUG,"11 DESCRIPTION NAPROSYN (naproxen) tablets, EC...","1 INDICATIONS AND USAGE NAPROSYN Tablets, EC-N...",12.1 Mechanism of Action Naproxen has analgesi...,"[0.010343536734580994, -0.021375570446252823, ...","[0.0074096317403018475, 0.01403836440294981, -...","[-0.006560009904205799, -0.0036851167678833008..."
36803,Hydrocodone Bitartrate and Homatropine Methylb...,HYDROCODONE BITARTRATE AND HOMATROPINE METHYLB...,HUMAN PRESCRIPTION DRUG,11 DESCRIPTION Hydrocodone bitartrate and homa...,1 INDICATIONS AND USAGE Hydrocodone bitartrate...,12.1 Mechanism of Action Hydrocodone Hydrocodo...,"[0.022935671731829643, -0.03710448741912842, -...","[0.02439817227423191, -0.014275806024670601, 0...","[0.020335186272859573, 0.007597180083394051, 0..."
36804,"dextroamphetamine saccharate, amphetamine aspa...","DEXTROAMPHETAMINE SACCHARATE, AMPHETAMINE ASPA...",HUMAN PRESCRIPTION DRUG,"11 DESCRIPTION Dextroamphetamine Saccharate, A...",1 INDICATIONS AND USAGE Dextroamphetamine Sacc...,12.1 Mechanism of Action Amphetamines are non-...,"[0.016110626980662346, -0.013666011393070221, ...","[-0.010057266801595688, -0.05343985930085182, ...","[-0.04683513194322586, -0.019234921783208847, ..."


In [9]:
favorite_drugs = ['naproxen', 'venlafaxine', 'botulinum']

# How many generic names match each pattern? How many are there of each?
generic_name_counts = {}
for favorite_drug in favorite_drugs:
    tag_flags = [favorite_drug.lower() in n.lower() for n in human_prescription_drugs_df['generic_name']]
    generic_name_counts[favorite_drug] = Counter(human_prescription_drugs_df.loc[ tag_flags, 'generic_name' ])

generic_name_counts

{'naproxen': Counter({'NAPROXEN': 149,
          'NAPROXEN SODIUM': 39,
          'NAPROXEN AND ESOMEPRAZOLE MAGNESIUM': 4,
          'SUMATRIPTAN AND NAPROXEN SODIUM': 3,
          'SUMATRIPTAN SUCCINATE AND NAPROXEN SODIUM': 2,
          'NAPROXEN , CAPSAICIN': 1,
          'NAPROXEN ORAL': 1}),
 'venlafaxine': Counter({'VENLAFAXINE HYDROCHLORIDE': 134,
          'VENLAFAXINE': 50,
          'DESVENLAFAXINE SUCCINATE': 21,
          'DESVENLAFAXINE': 19,
          'DESVENLAFAXINE ER': 2,
          'VENLAFAXINE HCL ER': 2,
          'VENLAFAXINE HCL': 2,
          'VENLAFAXINE HYDROCHLORIDE, EXTENDED RELEASE': 1}),
 'botulinum': Counter({'BOTULINUM TOXIN TYPE A': 3,
          'INCOBOTULINUMTOXINA': 2,
          'ONABOTULINUMTOXINA': 2,
          'HUMAN BOTULINUM NEUROTOXIN A/B IMMUNE GLOBULIN': 1,
          'PRABOTULINUM TOXIN TYPE A': 1,
          'LETIBOTULINUMTOXINA-WLBG': 1,
          'RIMABOTULINUMTOXINB': 1})}

# Cluster by three different embeddings

In [10]:
from collections import Counter;

def add_cluster_cols(df, embedding_col='description', prefix='cluster', letters='ABCDE', max_threshold=1):
    from scipy.cluster.hierarchy import ward, fcluster
    from scipy.spatial.distance import pdist
    import math

    # cluster the sentence vectors at various levels
    X = df[embedding_col].tolist()
    y = pdist(X, metric='cosine')
    z = ward(y)

    for i in range(len(letters)):
        letter = letters[i]
        col_name = f'{prefix}_{letter}'
        cluster_id = fcluster(z, max_threshold/2**i, criterion='distance')
        digits = 1 + math.floor(math.log10(max(cluster_id)))
        df[col_name] = [letter + str(cid).zfill(digits) + f'_{prefix}' for cid in cluster_id]

    cluster_cols = [c for c in df.columns if c.startswith(f'{prefix}_')]
    return df.sort_values(by=cluster_cols)


In [ ]:
CLUSTERED_FILE = "human_prescription_drugs_clustered_v2.parquet"

TARGETS = ["description", "indications_and_usage", "mechanism_of_action"]

if os.path.exists(CLUSTERED_FILE):
   human_prescription_drugs_df = pd.read_parquet(CLUSTERED_FILE)
else:  # 27 min

   for target in TARGETS:
      human_prescription_drugs_df = add_cluster_cols(human_prescription_drugs_df, 
                                                      embedding_col=f"{target}_vector", 
                                                      prefix=f"{target}_cluster", 
                                                      letters='ABCDEFGHI', 
                                                      max_threshold=16)

   human_prescription_drugs_df.to_parquet(CLUSTERED_FILE)

In [12]:
human_prescription_drugs_df.columns
# TARGETS[0]

Index(['brand_name', 'generic_name', 'product_type', 'description',
       'indications_and_usage', 'mechanism_of_action', 'description_vector',
       'indications_and_usage_vector', 'mechanism_of_action_vector',
       'description_cluster_A', 'description_cluster_B',
       'description_cluster_C', 'description_cluster_D',
       'description_cluster_E', 'description_cluster_F',
       'description_cluster_G', 'description_cluster_H',
       'description_cluster_I', 'indications_and_usage_cluster_A',
       'indications_and_usage_cluster_B', 'indications_and_usage_cluster_C',
       'indications_and_usage_cluster_D', 'indications_and_usage_cluster_E',
       'indications_and_usage_cluster_F', 'indications_and_usage_cluster_G',
       'indications_and_usage_cluster_H', 'indications_and_usage_cluster_I',
       'mechanism_of_action_cluster_A', 'mechanism_of_action_cluster_B',
       'mechanism_of_action_cluster_C', 'mechanism_of_action_cluster_D',
       'mechanism_of_action_cluster_E

In [13]:
target = TARGETS[0]
cluster_names = [c for c in sorted(human_prescription_drugs_df.columns) if c.startswith(f"{target}_cluster_")]
cluster_names

['description_cluster_A',
 'description_cluster_B',
 'description_cluster_C',
 'description_cluster_D',
 'description_cluster_E',
 'description_cluster_F',
 'description_cluster_G',
 'description_cluster_H',
 'description_cluster_I']

In [14]:
subcluster_counts = {}
for target in TARGETS:
    subcluster_counts[target] = {}
    cluster_names = [c for c in sorted(human_prescription_drugs_df.columns) if c.startswith(f"{target}_cluster_")]
    for cluster_col in cluster_names:
        num_clusters = len(Counter(human_prescription_drugs_df[cluster_col]))
        subcluster_counts[target][cluster_col] = num_clusters

subcluster_counts

{'description': {'description_cluster_A': 6,
  'description_cluster_B': 35,
  'description_cluster_C': 137,
  'description_cluster_D': 389,
  'description_cluster_E': 809,
  'description_cluster_F': 1550,
  'description_cluster_G': 2960,
  'description_cluster_H': 4281,
  'description_cluster_I': 5600},
 'indications_and_usage': {'indications_and_usage_cluster_A': 9,
  'indications_and_usage_cluster_B': 41,
  'indications_and_usage_cluster_C': 144,
  'indications_and_usage_cluster_D': 411,
  'indications_and_usage_cluster_E': 985,
  'indications_and_usage_cluster_F': 2168,
  'indications_and_usage_cluster_G': 3856,
  'indications_and_usage_cluster_H': 4836,
  'indications_and_usage_cluster_I': 5713},
 'mechanism_of_action': {'mechanism_of_action_cluster_A': 8,
  'mechanism_of_action_cluster_B': 26,
  'mechanism_of_action_cluster_C': 83,
  'mechanism_of_action_cluster_D': 248,
  'mechanism_of_action_cluster_E': 519,
  'mechanism_of_action_cluster_F': 982,
  'mechanism_of_action_cluster_

In [15]:
level_A_counts = {}
for target in TARGETS:
    level_A_counts[target] = Counter(human_prescription_drugs_df[f'{target}_cluster_A'])


level_A_counts

{'description': Counter({'A2_description_cluster': 15168,
          'A4_description_cluster': 10242,
          'A6_description_cluster': 3526,
          'A5_description_cluster': 3088,
          'A3_description_cluster': 2969,
          'A1_description_cluster': 1813}),
 'indications_and_usage': Counter({'A9_indications_and_usage_cluster': 8220,
          'A7_indications_and_usage_cluster': 6748,
          'A4_indications_and_usage_cluster': 4741,
          'A5_indications_and_usage_cluster': 4222,
          'A3_indications_and_usage_cluster': 4028,
          'A2_indications_and_usage_cluster': 2683,
          'A8_indications_and_usage_cluster': 2473,
          'A6_indications_and_usage_cluster': 2195,
          'A1_indications_and_usage_cluster': 1496}),
 'mechanism_of_action': Counter({'A1_mechanism_of_action_cluster': 17238,
          'A4_mechanism_of_action_cluster': 5264,
          'A2_mechanism_of_action_cluster': 3248,
          'A8_mechanism_of_action_cluster': 3088,
          

In [16]:
non_blank_fractions = {}
for target in TARGETS:
    non_blank = sum(human_prescription_drugs_df[target] != '') # 34993
    total = len(human_prescription_drugs_df) # 36806
    non_blank_fractions[target] = non_blank/total

non_blank_fractions

{'description': 0.9507417268923545,
 'indications_and_usage': 0.9593544530783025,
 'mechanism_of_action': 0.5316524479704396}

## Sort the drugs by each of the three text columns (TARGETS)

Here I am sorting by the subclusters for each target. Alternatively, you could sort them in Excel, but you need to be sure to sort by all of the subclusters for a give target in order from large to small.

I write out the sorted data without the vectors, because vectors make the files too big.

In [17]:
def sort_by_target(df, target):
    print(f"sorting by '{target}'")
    cluster_names = [c for c in sorted(df.columns) if c.startswith(f"{target}_cluster_")]
    sorted_by_target = df[df[target] != ''].sort_values(cluster_names)
    return sorted_by_target

sorted_by = {}
for target in TARGETS:
    sorted_by[target] = sort_by_target(human_prescription_drugs_df, target)

sorting by 'description'
sorting by 'indications_and_usage'
sorting by 'mechanism_of_action'


In [18]:
for target in TARGETS:
       df = sorted_by[target]
       base_cols = ['brand_name', 'generic_name', 'description', 'indications_and_usage', 'mechanism_of_action']
       cluster_cols = [c for c in df if f"{target}_cluster" in c and 'vector' not in c]
       df[[*base_cols, *cluster_cols]].to_excel(f"human_prescription_drugs_sorted_by_{target}.xlsx")
